本教程的这一部分介绍了面向具有凸面优化高级知识的用户的CVXPY功能。我们推荐Boyd和Vandenberghe的[凸优化](http://www.stanford.edu/~boyd/cvxbook/)作为您不熟悉的任何术语的参考。

# 双变量

您可以使用CVXPY查找问题的最优双变量。当您调用`prob.solve()`时，解决方案中的每个双变量都存储在它所对应的约束的`dual_value`字段中。

In [1]:
import cvxpy as cvx

# Create two scalar optimization variables.
x = cvx.Variable()
y = cvx.Variable()

# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

# Form objective.
obj = cvx.Minimize((x - y)**2)

# Form and solve problem.
prob = cvx.Problem(obj, constraints)
prob.solve()

# The optimal dual variable (Lagrange multiplier) for
# a constraint is stored in constraint.dual_value.
print("optimal (x + y == 1) dual variable", constraints[0].dual_value)
print("optimal (x - y >= 1) dual variable", constraints[1].dual_value)
print("x - y value:", (x - y).value)

optimal (x + y == 1) dual variable 2.0
optimal (x - y >= 1) dual variable 2.0
x - y value: 1.0


`x - y >= 1`的双变量是2。通过互补性，这意味着`x - y`是1，我们可以看到这是正确的。事实上双变量非零也告诉我们，如果我们收紧`x - y >= 1`（即增加右边），问题的最优值将会增加。

# 属性

可以使用指定其他属性的属性创建变量和参数。 例如，`Variable(nonneg=True)`是一个被限制为非负的标量变量。 类似地，`Parameter(nonpos=True)`是一个被限制为非正的标量参数。 Leaf的完整构造函数（变量和参数的父类）如下所示。

The value field of Variables and Parameters can be assigned a value after construction, but the assigned value must satisfy the object attributes. A Euclidean projection onto the set defined by the attributes is given by the project method.

变量和参数的`value`字段可以在构建后赋值，但分配的值必须满足对象属性。由投影方法给出由属性定义的集合上的欧几里德投影。

In [2]:
p = cvx.Parameter(nonneg=True)
try:
    p.value = -1
except Exception as e:
    print(e)

print("Projection:", p.project(-1))

Parameter value must be nonnegative.
Projection: 0.0


为叶赋值的一个明智习惯是`leaf.value = leaf.project(val)`，以确保赋值满足叶的属性。一个稍微有效的变体是`leaf.project_and_assign(val)`，它直接投影和赋值，而不另外检查该值是否满足叶的属性。在大多数情况下，投影和检查值满足叶的属性是廉价操作（即O(n)），但是对于对称正半定或负半定叶，这些操作计算特征值分解。

许多属性，如非负性和对称性，都可以通过约束轻松指定。在变量中指定属性有什么优势？主要优点是指定属性可实现更细粒度的DCP分析。例如，通过`x = Variable(nonpos=True)`创建变量`x`，通知DCP分析器`x`是非正的。通过`x = Variable()`创建变量`x`并单独添加约束`x >= 0`则不会提供任何关于`x`的符号信息到DCP分析器。

# 半定矩阵

许多凸优化问题都涉及到正或负半定矩阵约束(即，SDPs)。您可以通过两种方式在CVXPY中执行此操作。第一种方法是使用`Variable((n, n), PSD=True)`创建一个n乘n变量，约束为对称半正定。 例如，

In [3]:
# Creates a 100 by 100 positive semidefinite variable.
X = cvx.Variable((100, 100), PSD=True)

# You can use X anywhere you would use
# a normal CVXPY variable.
obj = cvx.Minimize(cvx.norm(X) + cvx.sum(X))

第二种方法是使用`>>`或`<<`运算符创建一个半正定锥约束。如果`X`和`Y`是`n*n`变量，则约束`X >> Y`意味着对于所有$z \in \mathcal{R}^n$，$z^T(X - Y)z \geq 0$。 换句话说$X + X^T$是半正定的。约束不要求`X`和`Y`是对称的。正半定锥约束的两边必须是平方矩阵和仿射。

以下代码显示如何将矩阵表达式约束为正半负或负半确定（但不一定对称）。

为了限制矩阵表达式是对称的，只需写

You can also use Variable((n, n), symmetric=True) to create an n by n variable constrained to be symmetric. The difference between specifying that a variable is symmetric via attributes and adding the constraint X == X.T is that attributes are parsed for DCP information and a symmetric variable is defined over the (lower dimensional) vector space of symmetric matrices.

你也可以使用`Variable((n, n), symmetric=True)`来创建一个`n*n`的变量，约束为对称的。指定变量对称通过属性定义与添加约束`X == X.T`的区别在于：DCP信息解析属性，并在对称矩阵的（低维）向量空间上定义对称变量。

# 混合整数程序

在混合整数程序中，某些变量被约束为布尔值（即0或1）或整数值。可以通过创建仅有布尔或整数值条目的属性的变量来构造混合整数程序：

# 复数值表达式

默认情况下，变量和参数是真数。复数值变量和参数可以通过设置属性`complex=True`来创建。 同样，通过设置属性`imag=True`可以创建纯虚构的变量和参数。包含复数变量，参数或常量的表达式可能是复数值。函数`is_real`，`is_complex`和`is_imag`分别返回一个表达式是纯粹真实的，复数的还是纯虚的。

In [4]:
# A complex valued variable.
x = cvx.Variable(complex=True)
# A purely imaginary parameter.
p = cvx.Parameter(imag=True)

print("p.is_imag() = ", p.is_imag())
print("(x + 2).is_real() = ", (x + 2).is_real())

p.is_imag() =  True
(x + 2).is_real() =  False


问题目标和不等式约束中的顶级表达式必须是实值的，但子表达式可能是复值。算术和所有线性原子可以定义复值表达式。所有非线性原子`abs`以及除`norm(X, p)`(`p < 1`)的所有`norms`都可用于复值表达式。所有域是对称矩阵的原子定义为Hermitian矩阵。类似地，为复数`x`和`Hermitian P`定义了原子`quad_form(x, P)`和`matrix_frac(x, P)`。最后，为复数表达式定义了等式和正定半定义约束。

提供以下附加原子用于处理复杂表达式：

+ `real(expr)` 给出了`expr`的真值部分
+ `imag(expr)` 给出了`expr`的虚部(即：`expr = real(expr) + 1j*imag(expr)`)
+ `conj(expr)` 给出`expr`的复共轭
+ `expr.H` 给出`expr`的Hermitian(共轭)转置


# 变换

变换提供了操作原子函数之外CVXPY对象的其他方法。例如，`indicator`变换将约束列表转换为表示凸函数的表达式，该函数在约束条件成立时取值0，违反条件时取$\infty$。

In [5]:
x = cvx.Variable()
constraints = [0 <= x, x <= 1]
expr = cvx.transforms.indicator(constraints)
x.value = .5
print("expr.value = ", expr.value)
x.value = 2
print("expr.value = ", expr.value)

expr.value =  0
expr.value =  0


[Transforms](http://www.cvxpy.org/api_reference/cvxpy.transforms.html#transforms-api)中讨论了可用的全套转换。

# 问题算术

为了方便起见，算术运算对于问题和目标已经超载。问题算术很有用，因为它可以让你将问题写成一个小问题的总和。下面给出了增加，减少和乘以目标的规则。

增加和增加问题的规则同样简单：

请注意，`+`运算符连接约束列表，因为这是Python列表的默认行为。就地操作符`+=`，`-=`和`*=`也支持目标和问题，并遵循上述相同的规则。

# 求解方法选项

`solve`方法带有可选参数，可以让您更改CVXPY如何求解问题。这是求解方法的签名：

我们将在下面详细讨论可选参数。

## 选择求解器

CVXPY与开源求解器[ECOS](https://www.embotech.com/ECOS)，[ECOS_BB](https://github.com/embotech/ecos#mixed-integer-socps-ecos_bb)，[OSQP](http://osqp.readthedocs.io/)和[SCS](http://github.com/cvxgrp/scs)一起发布。如果单独安装，许多其他求解器可以由CVXPY调用。 下表显示了支持的求解器可以处理的问题类型。

![支持的求解器](solver.png)

这里EXP指的是指数圆锥约束的问题。 指数锥被定义为$$\{(x,y,z) \mid y > 0, y\exp(x/y) \leq z \} \cup \{ (x,y,z) \mid x \leq 0, y = 0, z \geq 0\}$$。

您不能在CVXPY中明确指定圆锥约束，但当CVXPY将问题转换为标准形式时，会添加圆锥约束。

默认情况下，CVXPY调用最专门针对问题类型的求解器。例如，SOCPs调用ECOS。 SCS可以处理所有问题（混合整数程序除外）。混合整数LP和SOCP调用ECOS_BB。如果问题是QP，CVXPY将使用OSQP。

您可以使用solver关键字参数来更改由CVXPY调用的求解器。 如果您选择的求解器无法解决问题，CVXPY将引发异常。 以下是使用不同解算器解决相同问题的示例代码。

In [6]:
# 当本地没有安装相应求解器时，以下部分可能触发异常

# Solving a problem with different solvers.
x = cvx.Variable(2)
obj = cvx.Minimize(x[0] + cvx.norm(x, 1))
constraints = [x >= 2]
prob = cvx.Problem(obj, constraints)

# Solve with OSQP.
prob.solve(solver=cvx.OSQP)
print(("optimal value with OSQP:", prob.value))

# Solve with ECOS.
prob.solve(solver=cvx.ECOS)
print(("optimal value with ECOS:", prob.value))

# Solve with ECOS_BB.
prob.solve(solver=cvx.ECOS_BB)
print(("optimal value with ECOS_BB:", prob.value))

# Solve with CVXOPT.
prob.solve(solver=cvx.CVXOPT)
print(("optimal value with CVXOPT:", prob.value))

# Solve with SCS.
#prob.solve(solver=cvx.SCS)
#print(("optimal value with SCS:", prob.value))

# Solve with GLPK.
prob.solve(solver=cvx.GLPK)
print(("optimal value with GLPK:", prob.value))

# Solve with GLPK_MI.
prob.solve(solver=cvx.GLPK_MI)
print(("optimal value with GLPK_MI:", prob.value))

# Solve with GUROBI.
#prob.solve(solver=cvx.GUROBI)
#print(("optimal value with GUROBI:", prob.value))

# Solve with MOSEK.
#prob.solve(solver=cvx.MOSEK)
#print(("optimal value with MOSEK:", prob.value))

# Solve with Elemental.
#prob.solve(solver=cvx.ELEMENTAL)
#print(("optimal value with Elemental:", prob.value))

# Solve with CBC.
#prob.solve(solver=cvx.CBC)
#print(("optimal value with CBC:", prob.value))

('optimal value with OSQP:', 6.0)
('optimal value with ECOS:', 5.999999995510096)
('optimal value with ECOS_BB:', 5.999999995510096)
('optimal value with CVXOPT:', 6.000000005119901)
('optimal value with GLPK:', 6.0)
('optimal value with GLPK_MI:', 6.0)


使用installed_solvers实用程序函数获取CVXPY支持的解决方案列表。

In [7]:
print(cvx.installed_solvers())

['ECOS', 'ECOS_BB', 'CVXOPT', 'GLPK', 'GLPK_MI', 'OSQP']


## 查看求解器输出

所有求解器可以在求解问题的同时打印进度的信息。这些信息可以用于调试求解器错误。要查看解算器的输出，请在solve方法中设置`verbose=True`。

In [8]:
# Solve with ECOS and display output.
prob.solve(solver=cvx.ECOS, verbose=True)
print("optimal value with ECOS:", prob.value)


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +6.667e-01  +7.067e-01  +6e+00  6e-01  1e-02  1e+00  9e-01    ---    ---    1  1  - |  -  - 
 1  +3.500e+00  +3.925e+00  +1e+00  3e-01  4e-03  8e-01  2e-01  0.9890  2e-01   1  1  1 |  0  0
 2  +5.716e+00  +5.825e+00  +2e-01  6e-02  8e-04  2e-01  4e-02  0.9091  8e-02   1  1  1 |  0  0
 3  +5.997e+00  +5.998e+00  +3e-03  7e-04  1e-05  2e-03  5e-04  0.9881  1e-04   1  1  1 |  0  0
 4  +6.000e+00  +6.000e+00  +3e-05  8e-06  1e-07  3e-05  5e-06  0.9890  1e-04   1  1  1 |  0  0
 5  +6.000e+00  +6.000e+00  +3e-07  9e-08  1e-09  3e-07  6e-08  0.9890  1e-04   1  0  0 |  0  0
 6  +6.000e+00  +6.000e+00  +4e-09  1e-09  1e-11  3e-09  6e-10  0.9890  1e-04   1  0  0 |  0  0

OPTIMAL (within feastol=9.9e-10, reltol=6.2e-10, abstol=3.7e-09).
Runtime: 0.000741 seconds.

optimal value with ECOS: 5.999999995510096


## 求解统计

当在问题对象上调用求解方法时，调用求解器，问题对象将记录最优值，原始和双重变量的值以及多个求解器统计量。我们已经讨论过如何查看最佳值和变量值。求解器统计信息通过`problem.solver_stats`属性访问，该属性返回`SolverStats`对象。例如，`problem.solver_stats.solve_time`给出求解器求解的时间。

## 热启动

当为多个参数值解决相同的问题时，许多解算器可以利用以前解决方案中的工作（即热启动）。例如，解算器可能会将以前的解决方案用作初始点或重新使用缓存的矩阵分解。 默认启用热启动，并使用`warm_start`求解器选项进行控制。下面的代码显示了热启动如何加速求解一系列相关的最小二乘问题。

In [9]:
import cvxpy as cvx
import numpy

# Problem data.
m = 2000
n = 1000
numpy.random.seed(1)
A = numpy.random.randn(m, n)
b = cvx.Parameter(shape=(m,))

# Construct the problem.
x = cvx.Variable(shape=(n,))
prob = cvx.Problem(cvx.Minimize(cvx.sum_squares(A * x - b)), [x >= 0])

b.value = numpy.random.randn(m)
prob.solve()
print(("First solve time:", prob.solver_stats.solve_time))

b.value = numpy.random.randn(m)
prob.solve(warm_start=True)
print(("Second solve time:", prob.solver_stats.solve_time))

('First solve time:', 25.75594788)
('Second solve time:', 1.875382336)


在这种情况下，加速来自缓存KKT矩阵分解。如果`A`是一个参数，分解缓存将不可能，而热启动的好处只是一个很好的初始点。

## 设置求解器选项

对于OSQP，ECOS，ECOS_BB，MOSEK，CBC，CVXOPT和SCS，Python接口允许您设置求解器选项，如最大迭代次数。您可以通过CVXPY作为关键字参数传递这些选项。

例如，这里我们告诉SCS使用间接方法求解线性方程而不是直接方法。

In [11]:
# Solve with SCS, use sparse-indirect method.
#prob.solve(solver=cvx.SCS, verbose=True, use_indirect=True)
#print("optimal value with SCS:", prob.value)

此处省略翻译。参见[详细选项列表](http://www.cvxpy.org/tutorial/advanced/index.html#warm-start)

# 获得标准格式

如果您有兴趣获取CVXPY针对问题生成的标准表单，则可以使用`get_problem_data`方法。 在问题对象上调用`get_problem_data(solver)`会返回CVXPY传递给解算器的参数的字典。如果您选择的求解器无法解决问题，CVXPY将引发异常。

In [12]:
# Get OSQP arguments.
#data = prob.get_problem_data(cvx.OSQP)

# Get ECOS arguments.
data = prob.get_problem_data(cvx.ECOS)

# Get ECOS_BB arguments.
#data = prob.get_problem_data(cvx.ECOS_BB)

# Get CVXOPT arguments.
#data = prob.get_problem_data(cvx.CVXOPT)

# Get SCS arguments.
#data = prob.get_problem_data(cvx.SCS)

In [13]:
data

({'c': array([1., 0., 0., ..., 0., 0., 0.]),
  'offset': array([0.]),
  'dims': <cvxpy.reductions.solvers.conic_solvers.conic_solver.ConeDims at 0x7fda32cfbe10>,
  'A': None,
  'b': None,
  'G': <3003x1001 sparse matrix of type '<class 'numpy.float64'>'
  	with 2001002 stored elements in Compressed Sparse Column format>,
  'h': array([ 1.        , -0.        , -0.        , ...,  2.0726169 ,
         -3.6091971 ,  0.96813862])},
 [[],
  (),
  {'var_id': 177,
   'offset': 0.0,
   'eq_constr': [],
   'other_constr': [NonPos(Expression(AFFINE, NONPOSITIVE, (1,))),
    NonPos(Expression(AFFINE, UNKNOWN, (1000,))),
    SOC(Expression(AFFINE, UNKNOWN, (1,)))]}])

求解标准圆锥形问题后，由`get_problem_data`返回结果，可以使用`unpack_results`方法解压缩原始求解器输出。在一个问题上调用`unpack_results(solver, solver_output)`将更新所有原始和双重变量的值以及问题值和状态。

例如，以下代码等同于直接使用CVXPY解决问题：

In [14]:
# Get ECOS arguments.
data = prob.get_problem_data(cvx.ECOS)
# Call ECOS solver.
solver_output = ecos.solve(data["c"], data["G"], data["h"],
                           data["dims"], data["A"], data["b"])
# Unpack raw solver output.
prob.unpack_results(cvx.ECOS, solver_output)

NameError: name 'ecos' is not defined

# Reductions

CVXPY使用reductions系统将用户提出的问题形式重新写入解算器将接受的标准形式。reduction是从一个问题向同等问题的转变。如果一个解决方案可以有效地转换为另一个解决方案，那么两个问题是等价的。reduction将CVXPY问题作为输入并输出CVXPY问题。[Reductions](http://www.cvxpy.org/api_reference/cvxpy.reductions.html#reductions-api)中讨论了可用的全套Reductions。